## Experiments with Panel Widgets

In [ ]:
import panel as pn

In [ ]:
pn.extension()

### Radio buttons select a text entry or a slider

In [ ]:
budget_levels = [
    ('$0', 0, 0),
    ('$1M', 1000000, 100000),
    ('$2.5M', 2500000, 250000),
    ('$5M', 5000000, 500000),
    ('$10M', 10000000, 1000000),
    ('$25M', 25000000, 2500000),
    ('$50M', 50000000, 5000000),
    ('$100M', 100000000, 10000000),
]

In [ ]:
class BudgetBox(pn.Column):

    def __init__(self):
        super(BudgetBox, self).__init__(margin=(15,0,15,5))
        self.budget_type = pn.widgets.RadioBoxGroup(options=['Return on Investment','Single Budget'], inline=False)
        self.budget_type.param.watch(self.budget_type_cb, ['value'])
        self.levels = [x[0] for x in budget_levels]
        self.map = { x[0]: x[1:] for x in budget_levels }
        self.slider = pn.widgets.DiscreteSlider(
            name='Maximum Budget',
            options = self.levels[:1],
            value = self.levels[0]
        )
        self.slider.param.watch(self.slider_cb, ['value'])
        # self.slider_box = pn.WidgetBox(self.slider)
        self.float_input = pn.widgets.FloatInput(name='Budget', value=0, format='$0,0.00')
        self.row = pn.Row(self.budget_type, self.slider, 'display placeholder')
        self.append(self.row)
        # self.append('display placeholder')

    def budget_type_cb(self, args):
        w = self.float_input if self.budget_type.value.startswith('Single') else self.slider
        self.row[1] = w
        self.row[2] = 'widget changed to ' + self.budget_type.value

    def slider_cb(self, args):
        b, x = self.map[self.slider.value]
        self.row[-1] = f'0 to {b} in steps of {x}'

    def set_budget_max(self, n):
        for i in range(len(budget_levels)-1, -1, -1):
            if n >= budget_levels[i][1]:
                self.slider.options = self.levels[:i+1]
                break


In [ ]:
w = BudgetBox()

In [ ]:
w

In [ ]:
w.set_budget_max(6000000)

### Tabs

In [ ]:
class BudgetBox(pn.Column):

    levels = [
        ('$0', 0),
        ('$500K', 500000),
        ('$1M', 1000000),
        ('$2.5M', 2500000),
        ('$5M', 5000000),
        ('$10M', 10000000),
        ('$25M', 25000000),
        ('$50M', 50000000),
        ('$100M', 100000000),
    ]

    boxh = 125
    boxw = 400
    npts = 10

    def __init__(self):
        super(BudgetBox, self).__init__(margin=(15,0,15,5))
        self.labels = [x[0] for x in BudgetBox.levels]
        self.map = { x[0]: x[1] for x in BudgetBox.levels }
        self.tabs = pn.Tabs(
            ('ROI', 
             pn.WidgetBox(
                pn.widgets.DiscreteSlider(
                    options = self.labels[:1], 
                    value = self.labels[0],
                    name = 'Maximum Budget'
                ),
                height=BudgetBox.boxh,
                width=BudgetBox.boxw)),
            ('Fixed', 
             pn.WidgetBox(
                pn.widgets.TextInput(name='Budget Amount', value='$'),
                height=BudgetBox.boxh,
                width=BudgetBox.boxw)),
            ('ℹ️',
            pn.WidgetBox(
                pn.pane.Markdown('''
                    **ROI** (return on investment) will generate a plot that shows the optimal set
                    of gates for several different budget levels.

                    **Fixed** will compute the optimal set of gates for a single budget.
                '''),
                height=BudgetBox.boxh,
                width=BudgetBox.boxw))
        )
        self.append(self.tabs)
        self.slider = self.tabs[0][0]
        self.input = self.tabs[1][0]

    def set_budget_max(self, n):
        for i in range(len(BudgetBox.levels)-1, -1, -1):
            if n >= BudgetBox.levels[i][1]:
                self.slider.options = self.labels[:i+1]
                break

    def values(self):
        if self.tabs.active == 0:
            x = self.map[self.slider.value]
            return x, (x // BudgetBox.npts)
        else:
            s = self.input.value
            if s.startswith('$'):
                s = s[1:]
            return self.input.value, 0


In [ ]:
w = BudgetBox()

In [ ]:
w

In [ ]:
w.tabs[1]

In [ ]:
w.tabs[1] = pn.pane.HTML('<p>hi</p>')

In [ ]:
dir(w.tabs)

In [ ]:
w.tabs.objects

In [ ]:
w.set_budget_max(1000000)

In [ ]:
w.values()

In [ ]:
w = pn.widgets.DiscreteSlider(options = ['a','b','c'], value = 'b', name='Letters')

In [ ]:
w

In [ ]:
w.value

In [ ]:
w.name

In [ ]:
from babel.numbers import parse_number, parse_decimal

In [ ]:
parse_decimal('12,34.8', locale='en_US')

In [ ]:
def parse_dollar_amount(s):
    try:
        if s.startswith('$'):
            s = s[1:]
        if s.endswith(('K','M')):
            multiplier = 1000 if s.endswith('K') else 1000000
            res = int(float(s[:-1]) * multiplier)
        elif ',' in s:
            parts = s.split(',')
            assert len(parts[0]) <= 3 and (len(parts) == 1 or all(len(p) == 3 for p in parts[1:]))
            res = int(''.join(parts))
        else:
            res = int(s)
        return res
    except Exception:
        raise ValueError('unexpected format in dollar amount')

In [ ]:
parse_dollar_amount('1567')

In [ ]:
parse_dollar_amount('$1234')

In [ ]:
parse_dollar_amount('$500K')

In [ ]:
parse_dollar_amount('$123,456')

In [ ]:
parse_dollar_amount('12,345')

In [ ]:
parse_dollar_amount('$1.5M')

In [ ]:
# parse_dollar_amount('123,45')

### Welcome Text

In [ ]:
text = open('../static/welcome.html').read()

In [ ]:
pn.pane.HTML(text, width=600)

### Download Dialog

Display download options and a download button in a separate tab in the top level GUI.

In [ ]:
NB = 'Net benefit plot'
IT = 'Individual target plots'
BS = 'Budget summary table'
BD = 'Barrier detail table'

In [ ]:
boxes = pn.widgets.CheckBoxGroup(
    name='Select items to download:',
    inline=False,
    options=[NB, IT, BS, BD],
    value = [NB, IT, BS, BD]
)

In [ ]:
text_input = pn.widgets.TextInput(name='Archive File Name', placeholder='.zip')

In [ ]:
button = pn.widgets.Button(name='Download')

In [ ]:
def cb(*args, **kwargs):
    print('clicked!')

In [ ]:
fd = pn.widgets.FileDownload(callback = cb, filename='foo.txt')

In [ ]:
pn.Column(
    pn.Row(boxes, text_input),
    button,
    fd
)

In [ ]:
fd.filename

In [ ]:
fd._clicks

In [ ]:
fd._clicks += 1

In [ ]:
fd.filename = 'bar.txt'

In [ ]:
boxes.values

In [ ]:
dir(fd)

### Path and Directory Operations

In [ ]:
from pathlib import Path, PurePath

In [ ]:
from shutil import rmtree, make_archive

In [ ]:
Path.cwd()

In [ ]:
Path.cwd() / 'test'

In [ ]:
d = Path.cwd() / 'test'

In [ ]:
Path.exists(d)

In [ ]:
Path.mkdir(d)

In [ ]:
Path.mkdir(d/"foo")

In [ ]:
Path.mkdir(d/'foo'/'bar')

In [ ]:
Path.mkdir(d/'foo'/'baz')

In [ ]:
rmtree(d)

In [ ]:
p = make_archive('foo', 'zip', d / 'foo')

In [ ]:
p

### Target Boxes

Add an IntInput widget before each target name so users can specify a weight for the target.

In [ ]:
class Target(pn.Row):

    def __init__(self, t):
        super(Target, self).__init__(margin=(10,0,10,5))
        w = pn.widgets.IntInput(value=1, step=1, start=1, end=10, width=50, height=32)
        self.append(w)
        b = pn.widgets.Checkbox(name=t)
        self.append(b)

In [ ]:
Target('Fish habitat: Coho streams')

Use an IntSlider instead of the IntInput

In [ ]:
class Target(pn.Row):

    def __init__(self, t):
        super(Target, self).__init__(margin=(10,0,10,5))
        b = pn.widgets.Checkbox(name=t)
        self.append(b)
        w = pn.widgets.IntSlider(name='Weight', value=1, step=1, start=1, end=10, width=70)
        self.append(w)

Display the region name as the slider name

In [ ]:
class Target(pn.Row):

    def __init__(self, t):
        super(Target, self).__init__(margin=(10,0,10,5))
        w = pn.widgets.IntSlider(name=t, value=0, step=1, start=0, end=10, width=70)
        self.append(w)

In [7]:
col = pn.Column()
for s in ['Fish habitat: Coho streams', 'Fish habitat: Chinook streams', 'Fish habitat: Steelhead streams']:
    col.append(Target(s))
col

BokehModel(combine_events=True, render_bundle={'docs_json': {'988170e1-2a11-4f9d-91fc-6c046c4fbb07': {'defs': …